# 1. 데이터셋 확인

In [ ]:
# 데이터셋 클래스 확인

import os

def get_class_names(path):
    class_ids = set()

    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as f:
                    for line in f:
                        if line.strip():  # 빈 줄 방지
                            class_id = line.strip().split()[0]
                            class_ids.add(class_id)

    return sorted(list(class_ids))

# 사용 예시
PATH = 'Dataset/PATH'
class_names = get_class_names(PATH)
print(class_names)


['0']


In [ ]:
import os
from collections import defaultdict

# 라벨 존재 및 클래스 여부 파악
def analyze_label_files(path):
    class_counts = defaultdict(int)
    empty_file_count = 0
    total_label_files = 0

    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)
                total_label_files += 1
                with open(file_path, 'r') as f:
                    lines = [line.strip() for line in f.readlines() if line.strip()]
                    if not lines:
                        empty_file_count += 1  # 빈 txt 파일
                    else:
                        for line in lines:
                            class_id = line.split()[0]
                            class_counts[class_id] += 1

    # 출력
    print(f"[📂] 총 라벨 파일 수: {total_label_files}")
    print(f"[⭕] 빈 txt 파일 수 (정상): {empty_file_count}")
    print(f"[🔢] 클래스 개수: {len(class_counts)}")
    print(f"[📋] 클래스별 객체 수:")
    for class_id in sorted(class_counts.keys(), key=lambda x: int(x)):
        print(f"  - Class {class_id}: {class_counts[class_id]}개")

    return class_counts, empty_file_count

# 사용 예시
LABEL_PATH = 'Labels/PATH'
analyze_label_files(LABEL_PATH)

[📂] 총 라벨 파일 수: 354
[⭕] 빈 txt 파일 수 (정상): 169
[🔢] 클래스 개수: 1
[📋] 클래스별 객체 수:
  - Class 0: 541개


(defaultdict(int, {'0': 541}), 169)

In [ ]:
# 이미지 디렉토리와 라벨 디렉토리를 받아 YOLO 형식 라벨을 시각화하고, 잘못된 라벨 파일을 자동 검출하여 리스트로 반환
import os
import cv2
import matplotlib.pyplot as plt

def visualize_yolo_labels_and_check_errors(image_dir, label_dir, max_images=10):
    """
    Parameters:
        image_dir (str): 이미지 파일 경로 (jpg, png)
        label_dir (str): YOLO txt 라벨 경로
        max_images (int): 최대 시각화 이미지 수

    Returns:
        issues (list): 문제 있는 라벨 파일에 대한 설명 리스트
    """
    image_files = [f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.png'))]
    visualized = 0
    issues = []

    for img_name in sorted(image_files):
        image_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_dir, os.path.splitext(img_name)[0] + '.txt')

        img = cv2.imread(image_path)
        if img is None:
            issues.append(f"[READ ERROR] Cannot read image: {img_name}")
            continue

        h, w = img.shape[:2]
        if not os.path.exists(label_path):
            issues.append(f"[LABEL MISSING] No label file for: {img_name}")
            continue

        with open(label_path, 'r') as f:
            lines = f.readlines()

        if len(lines) == 0:
            issues.append(f"[EMPTY LABEL] Label file is empty: {img_name}")
            continue

        valid = True
        for line in lines:
            try:
                parts = line.strip().split()
                if len(parts) != 5:
                    raise ValueError("Expected 5 elements per line (class x y w h)")
                cls, xc, yc, bw, bh = map(float, parts)
                if not all(0 <= v <= 1 for v in [xc, yc, bw, bh]):
                    raise ValueError("Label values out of normalized range (0~1)")

                x1 = int((xc - bw / 2) * w)
                y1 = int((yc - bh / 2) * h)
                x2 = int((xc + bw / 2) * w)
                y2 = int((yc + bh / 2) * h)

                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, str(int(cls)), (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            except Exception as e:
                valid = False
                issues.append(f"[LABEL ERROR] {img_name}: {str(e)} → {line.strip()}")

        if valid and visualized < max_images:
            plt.figure(figsize=(10, 6))
            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f"Labeled: {img_name}")
            plt.axis('off')
            plt.show()
            visualized += 1

    return issues

IMG_PATH = 'Img/PATH'
LABEL_PATH = 'Label/PATH'
issues = visualize_yolo_labels_and_check_errors(IMG_PATH, LABEL_PATH)

# 문제 라벨 요약 출력
for issue in issues:
    print(issue)

In [ ]:
# 클래스별 이미지와 라벨 일치 여부 확인
import os

def list_relative_paths(base_dir, exts):
    result = []
    for root, _, files in os.walk(base_dir):
        for f in files:
            if any(f.lower().endswith(ext) for ext in exts):
                rel_path = os.path.relpath(os.path.join(root, f), base_dir)
                rel_path_no_ext = os.path.splitext(rel_path)[0]
                result.append(rel_path_no_ext)
    return sorted(result)

# 경로 설정
image_base_dir = "Img/PATH"
label_base_dir = "Label/PATH"

splits = ['train', 'val', 'test']
image_exts = ['.jpg', '.jpeg', '.png']
label_exts = ['.txt']

for split in splits:
    print(f"\n📂 Checking split: {split}")
    image_dir = os.path.join(image_base_dir, split)
    label_dir = os.path.join(label_base_dir, split)

    image_files = list_relative_paths(image_dir, image_exts)
    label_files = list_relative_paths(label_dir, label_exts)


    # 비교
    image_set = set(image_files)
    label_set = set(label_files)

    only_in_images = image_set - label_set
    only_in_labels = label_set - image_set

    if not only_in_images and not only_in_labels:
        print("✅ 이미지와 라벨 경로가 일치합니다.")
    else:
        print("❌ 불일치 항목 존재:")
        if only_in_images:
            print("  - 라벨 없음:", list(only_in_images)[:10], "...")
        if only_in_labels:
            print("  - 이미지 없음:", list(only_in_labels)[:10], "...")



📂 Checking split: train
✅ 이미지와 라벨 경로가 일치합니다.

📂 Checking split: val
✅ 이미지와 라벨 경로가 일치합니다.

📂 Checking split: test
✅ 이미지와 라벨 경로가 일치합니다.
